<a href="https://colab.research.google.com/github/rishikesh953/Theft-Detection/blob/main/Theft_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import statements

import torch
from torch.utils.data import Dataset
import torchvision.models as models
import cv2
import numpy as np
from skimage.transform import resize

In [ ]:
# preparing the dataset

class VideoDataset(Dataset):
    """Video dataset."""

    def __init__(self, data, timesep=30,rgb=3,h=120,w=120):

        self.datalocation = data
        self.timesep = timesep
        self.rgb = rgb
        self.h = h
        self.w = w

    def __len__(self):
        return self.datalocation.size()

    def __getitem__(self, index):
        if torch.is_tensor(index):
            idx = index.tolist()
        video = capture(self.dataloctions.iloc[index, 0],self.timesep,self.rgb,self.h,self.w)
        sample = {'video': torch.from_numpy(video), 'label': torch.from_numpy(np.asarray(self.dataloctions.iloc[idx, 1]))}
        return sample

def capture(filename,timesep,rgb,h,w):
    tmp = []
    frames = np.zeros((timesep,rgb,h,w), dtype=np.float)
    i=0
    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval , frame = vc.read()
    else:
        rval = False
    frm = resize(frame,(h, w,rgb))  
    frm = np.expand_dims(frm,axis=0)
    frm = np.moveaxis(frm, -1, 1)
    if(np.max(frm)>1):
        frm = frm/255.0
    frames[i][:] = frm
    i +=1
    while i < timesep:
        tmp[:] = frm[:]
        rval, frame = vc.read()
        frm = resize(frame,( h, w,rgb))
        frm = np.expand_dims(frm,axis=0)
        if(np.max(frm)>1):
            frm = frm/255.0
        frm = np.moveaxis(frm, -1, 1)
        frames[i-1][:] = frm # - tmp
        i +=1

    return frames


In [ ]:
import torchvision.models as models

model = models.densenet169(pretrained=True)

model = models.densenet169(pretrained=True).features

In [ ]:
import torchvision.models as models

#get only the conv layers from a pre-trained densenet169
model = models.densenet169(pretrained=True).features
# densenet169  contain 4 denseblock
#you can simply freez any of these by disable the grad from theme like these freez mean not to train these layers
# by default all layers are trainable
#freezing the first block
for param in model.denseblock1.parameters():
            param.requires_grad = False


Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


  0%|          | 0.00/54.7M [00:00<?, ?B/s]